In [1]:
filename = 'train.russian.v4_gold_conll'

In [5]:
corpus = []
with open(filename) as f:
  corpus = [line.split() for line in f if line.strip() and not line.startswith("#")]

In [3]:
import pandas as pd

In [6]:
df = pd.DataFrame(corpus)

In [4]:
!pip install stanza
import stanza
stanza.download('ru')
nlp = stanza.Pipeline('ru')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 691 kB 5.3 MB/s 
     |████████████████████████████████| 240 kB 35.4 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234927 sha256=61895c06e99908e413c82ca31eb3f9359664f3ec2f1062cfc2fd10b2bf77edeb
  Stored in directory: /root/.cache/pip/wheels/f3/e3/f2/1de1c2e3ed742e1df73e0f15d58864e50c7e64f607b548d6cf
Successfully built emoji


INFO:stanza:Downloading default packages for language: ru (Russian) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [7]:
def for_good_word(word, part_idx, ids, details, feats):
  if word.pos == 'VERB':
    ids.append('-')
    #all_verbs.append(word)
    #verb_ids.append(word.id)
    #events[word.lemma] = []
    details.append('VERB')
    feats.append(('-', '-', '-', '-'))
    
  elif word.deprel == 'obj' or word.deprel == 'nsubj':
    ids.append(part_idx)
    #all_participants.append(word)
    start_feats = ['-', '-', '-', '-']
    if word.feats:
      word_feats = word.feats.split('|')
      for feat in word_feats:
        if feat.split('=')[0] == 'Number':
          start_feats[0] = feat.split('=')[1]
        elif feat.split('=')[0] == 'Gender':
          start_feats[1] = feat.split('=')[1]
        elif feat.split('=')[0] == 'Person':
          start_feats[2] = feat.split('=')[1]
        elif feat.split('=')[0] == 'Animacy':
          start_feats[3] = feat.split('=')[1]
    feats.append(tuple(start_feats))
    if word.deprel == 'obj':
      details.append('obj')
    elif word.deprel == 'nsubj':
      details.append('nsubj')
      
  else:
    details.append('-')
    feats.append(('-', '-', '-', '-'))
    ids.append('-')
  return part_idx, ids, details, feats

In [9]:
def add_inf(doc, df1):
  lst = list(df1[3])
  # event chain for sentence
  #event_chain = []
  details = []
  feats = []
  counter = 0
  part_idx = 0
  ids = []
  for sentence in doc.sentences:
    #all_verbs = []
    #verb_ids = []
    #events = {}
    #all_participants = []
    for word in sentence.words:
      part_idx += 1
      if word.text == lst[counter]:
        counter += 1
        part_idx, ids, details, feats = for_good_word(word, part_idx, ids, details, feats)
      else:
        if (counter + 1) == len(lst):
          details.append('-')
          feats.append(('-', '-', '-', '-'))
          ids.append('-')
          break
        else:
          if word.text == lst[counter+1]:
            counter += 2
            details.append('-')
            feats.append(('-', '-', '-', '-'))
            ids.append('-')
            part_idx, ids, details, feats = for_good_word(word, part_idx, ids, details, feats)

  df1.loc[:, 'details'] = details
  df1.loc[:, 'ids'] = ids
  df1.loc[:, 'Number'] = [f[0] for f in feats]
  df1.loc[:, 'Gender'] = [f[1] for f in feats]
  df1.loc[:, 'Person'] = [f[2] for f in feats]
  df1.loc[:, 'Animacy'] = [f[3] for f in feats]
  return df1

In [11]:
def make_coref_table(df1):
  big_dict = {'num1': [], 'num2': [], 'gen1': [], 'gen2': [], 
              'pers1': [],'pers2': [], 'anim1': [], 'anim2': [], 
              'coref': [],}
  list_id = list(df1[df1['ids'] != '-'].ids)
  for id1 in list_id:
    for id2 in list_id[list_id.index(id1)+1:]:
      big_dict['num1'].append(list(df1[df1['ids'] == id1].Number)[0])
      big_dict['num2'].append(list(df1[df1['ids'] == id2].Number)[0])
      big_dict['gen1'].append(list(df1[df1['ids'] == id1].Gender)[0])
      big_dict['gen2'].append(list(df1[df1['ids'] == id2].Gender)[0])
      big_dict['pers1'].append(list(df1[df1['ids'] == id1].Person)[0])
      big_dict['pers2'].append(list(df1[df1['ids'] == id2].Person)[0])
      big_dict['anim1'].append(list(df1[df1['ids'] == id1].Animacy)[0])
      big_dict['anim2'].append(list(df1[df1['ids'] == id2].Animacy)[0])
      ref1 = list(df1[11][df1['ids'] == id1])[0]
      ref2 = list(df1[11][df1['ids'] == id2])[0]
      for ref in [ref1, ref2]:
        if ref.startswith('('):
          ref = ref[1:]
        if ref.startswith(')', -1):
          ref = ref[:-1]
      coref = 0
      if ref1 == ref2:
        coref = 1
      big_dict['coref'].append(coref)
  return big_dict

In [15]:
dic_list = []
for doc_id in list(df[0].unique()):
  try:
    df2 = add_inf(nlp(' '.join(df[df[0] == doc_id][3])), df[df[0] == doc_id])
    new_dic = make_coref_table(df2)
    dic_list.append(new_dic)
  except ValueError:
    continue

In [ ]:
pd.concat([pd.DataFrame(d) for d in dic_list], axis=0, join='outer', ignore_index=True)